In [6]:
!pip install tensorflow opencv-python mediapipe sklearn matplotlib

In [3]:
import cv2 
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [8]:
mp_holistic= mp.solutions.holistic      #Detection model
mp_drawing= mp.solutions.drawing_utils  #Drawing connections

In [9]:
def mediapipe_detection(image, model):
    image= cv2.cvtColor(image, cv2.COLOR_BGR2RGB)   #Color conversion
    image.flags.writeable= False                    #Image no longer writeable
    results= model.process(image)                   #Make prediction  
    image.flags.writeable= True                     #Image now writeable
    image= cv2.cvtColor(image, cv2.COLOR_RGB2BGR)   #Color conversion 
    return image, results

In [32]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks,mp_holistic.FACEMESH_CONTOURS)      #Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks,mp_holistic.POSE_CONNECTIONS)       #Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS)  #Draw left hand Connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS) #Draw right hand Connections

In [33]:
cap = cv2.VideoCapture(0)

#Access Mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        
        #read feed from cam(0)
        ret, frame= cap.read()
        
        #make detection
        image, results = mediapipe_detection(frame, holistic)
        
        #Draw connections on the frame
        draw_landmarks(image, results)
        
        #Output connection to the screen
        cv2.imshow('OpenCV Feed', image)
        
        #Exit Program with 'q'
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1